### Imports

In [1]:
import sys
import pandas as pd
import numpy as np
import gc

import scipy
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


import torch
import torch.nn as nn
from torch.utils.data import DataLoader

sys.path.append("scripts")
from scripts.review_dataloader import *

### GPU acceleration

In [2]:
if torch.cuda.is_available():
    print(f"Using CUDA version {torch.version.cuda}")
elif torch.backends.mps.is_available():
    print("Using MPS backend")
else:
    print("No backend detected, using CPU")

Using CUDA version 12.1


### Load dataset

In [ ]:
# review_dl = SteamReviewDataset("../data/reviews_100k.csv.bz2", shuffle=True)

reviews_df = pd.read_csv("../data/reviews_100k.csv.bz2", low_memory=False)
reviews_df["review"] = reviews_df["review"].astype(str)
reviews_df 

# optionally shuffle (games are in order!)
reviews_df = reviews_df.sample(frac=1).reset_index(drop=True)

# fewer data for testing
# 100k full corpus takes 5h+
# 10k takes 3 min
# 1k takes 1s
reviews_df = reviews_df[:10_000] 

print(f"loaded {len(reviews_df)} reviews")





loaded 10000 reviews


In [4]:
# Daten aufsplitten
x_train_raw, x_test_raw, y_train_raw, y_test_raw = train_test_split(reviews_df["review"], reviews_df["voted_up"], test_size=.33, random_state=42)   

# Kontrolle
print(f"y_train true/false ratio is {len(y_train_raw[y_train_raw == True])/len(y_train_raw):.2f}")
print(f"y_test true/false ratio is {len(y_test_raw[y_test_raw == True])/len(y_test_raw):.2f}")      


y_train true/false ratio is 0.69
y_test true/false ratio is 0.69


### Build feature representation

In [5]:
tfid_vectorizer = TfidfVectorizer(ngram_range=(1,3))

# ca. 45s
x_train_tfidf = tfid_vectorizer.fit_transform(x_train_raw) # document term matrix
x_test_tfidf = tfid_vectorizer.transform(x_test_raw)

print(tfid_vectorizer.get_feature_names_out()[:10]) # Vorschau

['ab' 'ab open' 'ab open window' 'ab rb' 'ab rb or' 'abandon'
 'abandon and' 'abandon and its' 'abandon and the' 'abandon any']


In [6]:
# How the data looks
# can't load whole dataset als dense matrix (1.5 TB!)
test: np.matrix = scipy.sparse.csr_matrix.todense(x_train_tfidf[2]) # from scipy.sparse.csr_matrix.todense(x_train)
print(type(test))
print(test.shape) # (1, 3317704)
np.count_nonzero(test)

<class 'numpy.matrix'>
(1, 561255)


114

### Configure PyTorch dataloader

In [7]:
class MinimalDataLoader(Dataset):
    def __init__(self, reviews, labels):
        self.reviews = reviews
        self.labels = np.array(y_train_raw)
        print(self.reviews.shape)
    
    def __len__(self):
        return self.reviews.shape[0]
    
    def __getitem__(self, idx):
        review = torch.tensor(scipy.sparse.csr_matrix.todense(self.reviews[idx]), dtype=torch.float32, device=torch.device('cuda:0')).squeeze(0)
        label = torch.tensor(self.labels[idx], dtype=torch.float32, device=torch.device('cuda:0'))
        return review, label   

In [8]:
train_data_provider = MinimalDataLoader(x_train_tfidf, y_train_raw)
print(len(train_data_provider))

(6700, 561255)
6700


In [9]:
train_data_provider.__getitem__(0)

(tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0'),
 tensor(0., device='cuda:0'))

### Defining the model

In [10]:
# model = nn.Sequential(
#     nn.Linear(train_data_provider.__getitem__(0)[0].shape[1], 1024),
#     nn.Dropout(0.1),
#     nn.Linear(1024, 512),
#     nn.Dropout(0.1),                                     
#     nn.Linear(512, 2),
#     nn.Sigmoid() # 0 .. 1
# )

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid() 
        )

    def forward(self, x):
        return self.model(x)

### Training

In [11]:
# num_epochs = 1

# data_it = iter(train_data_provider)

# for epoch in range(num_epochs):
#     data, label = next(data_it)
#     data = torch.tensor(data).float()
#     label = torch.tensor(label)
    
#     optimizer.zero_grad()
#     output = model.forward(data)
#     loss = criterion(output, label)
#     loss.backward()
#     optimizer.step()
    
#     # WIP

def train_model(train_loader, input_dim, num_epochs=10, lr=0.001, device="cuda:0"):
    model = BinaryClassifier(input_dim=input_dim).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        for batch_X, batch_y in progress_bar:
            optimizer.zero_grad()
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs} completed. Average Loss: {total_loss / len(train_loader):.4f}")
        
    return model

In [12]:
# Usage:
# Assuming X_train, y_train are tf-idf and labels (from train-test split)
batch_size = 32
input_dim = x_train_tfidf.shape[1]

# Create dataset and DataLoader
train_loader = DataLoader(train_data_provider, batch_size=batch_size, shuffle=True)

torch.cuda.empty_cache()
gc.collect()

# Train the model
trained_model = train_model(train_loader, input_dim=input_dim, num_epochs=5, lr=0.001)

Epoch 1/5: 100%|██████████| 210/210 [00:25<00:00,  8.22batch/s]


Epoch 1/5 completed. Average Loss: 0.4575


Epoch 2/5: 100%|██████████| 210/210 [00:25<00:00,  8.35batch/s]


Epoch 2/5 completed. Average Loss: 0.0458


Epoch 3/5: 100%|██████████| 210/210 [00:26<00:00,  7.95batch/s]


Epoch 3/5 completed. Average Loss: 0.0030


Epoch 4/5: 100%|██████████| 210/210 [00:26<00:00,  7.79batch/s]


Epoch 4/5 completed. Average Loss: 0.0009


Epoch 5/5: 100%|██████████| 210/210 [00:24<00:00,  8.58batch/s]

Epoch 5/5 completed. Average Loss: 0.0004
